In [1]:
!dir

 Volume in drive C has no label.
 Volume Serial Number is B8C5-A35A

 Directory of C:\Users\i.glavatskyi\Ironhack\Week_6\lab-spotify-api\code

13/10/2023  01:15    <DIR>          .
13/10/2023  01:15    <DIR>          ..
13/10/2023  01:14    <DIR>          .ipynb_checkpoints
13/10/2023  01:05                99 configspy.py
13/10/2023  01:13             7,121 lap_spotify_API-Copy1.ipynb
13/10/2023  01:15             7,121 lap_spotify_API.ipynb
               3 File(s)         14,341 bytes
               3 Dir(s)  172,938,182,656 bytes free


In [150]:
#from ..private import config_spotipy
import sys
from configspy import *
import spotipy
import json
import pandas as pd
import time

from spotipy.oauth2 import SpotifyClientCredentials


In [151]:
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=client_id, client_secret=client_secret))

In [152]:
def extract_song_id(uri):
    # Split the URI by ':'
    parts = uri.split(':')
    # The song ID is the last part
    song_id = parts[-1]
    return song_id


In [191]:
def get_audio_features(list_of_song_ids):
    audio_features = []
    
    try:
        for song_id in list_of_song_ids:
            # Get audio features for the song
            features = sp.audio_features(tracks=[song_id])
            audio_features.extend(features)
            #time.sleep(20)
            
    except Exception as e:
        print(f"Error: {e}")
    
    return audio_features

In [192]:
def extract_audio_feature_keys(features):
    # Extract the keys (audio feature names) from the first item in the list
    return list(features[0].keys())


In [193]:
# Function to search for a song and return its URIs (Spotify IDs)
def search_song(title, artist, limit=1):
    song_uris = []  # Initialize a list to store song URIs

    try:
        if artist:
            # Search for the song using both title and artist
            query = f"track:{title} artist:{artist}"
        else:
            # Search for the song using title alone
            query = f"track:{title}"

        # Add a delay before making the API call
        time.sleep(20)

        # Perform the search
        results = sp.search(q=query, type='track', limit=limit)

        # Check if any tracks were found
        if results['tracks']['items']:
            # Create a set to keep track of unique artist names
            unique_artists = set()
            
            for track in results['tracks']['items']:
                uri = track['uri']
                artist_name = track['artists'][0]['name']

                if artist_name not in unique_artists:
                    # Append the URI if it's the first instance of the artist
                    song_uris.append(uri)
                    unique_artists.add(artist_name)

        # Create a dictionary to store song data
        song_data = {
            'song_name': [],
            'artist': [],
            'spotify_id': []
        }

        for uri in song_uris:
            # Find the corresponding track in the search results
            track = next(item for item in results['tracks']['items'] if item['uri'] == uri)
            song_data['song_name'].append(track['name'])
            song_data['artist'].append(track['artists'][0]['name'])
            song_data['spotify_id'].append(extract_song_id(track['uri']))

        # Create a DataFrame from the song data
        song_df = pd.DataFrame(song_data)

    except Exception as e:
        # Handle exceptions, e.g., if there's an issue with the Spotify API
        print(f"Error: {e}")
        song_df = pd.DataFrame()

    return song_uris, song_df


In [194]:
audio_features_dict = {
        'song_name': [],
        'artist': [],
        'id': [],
        'danceability': [], 'energy': [], 'key': [], 'loudness': [],
        'mode': [], 'speechiness': [], 'acousticness': [], 'instrumentalness': [], 'liveness': [], 
        'valence': [], 'tempo': [], 'type': [], 'id': [], 'uri': [], 'track_href': [], 
        'analysis_url': [], 'duration_ms': [], 'time_signature': []
}

In [195]:
# Function to get song info with audio features
import pandas as pd

def get_song_info_with_features(title, artist, limit):
    # Search for song URIs and song_df
    song_uris, song_df = search_song(title, artist, limit)

    if not song_uris:
        return pd.DataFrame()  # Return an empty DataFrame if no song URIs are found

    # Retrieve audio features for all song URIs
    audio_features = get_audio_features(song_uris)

    # Convert the audio features into a DataFrame
    audio_features_df = pd.DataFrame(audio_features)

    # Concatenate the two DataFrames
    song_info_df = pd.concat([song_df, audio_features_df], axis=1)

    return song_info_df




In [196]:
#add to the existing dataset
def add_audio_features(df, audio_features_df):
    # Check if both dataframes have the same length
    if len(df) != len(audio_features_df):
        raise ValueError("Dataframes must have the same length.")

    # Concatenate the dataframes horizontally
    extended_df = pd.concat([df, audio_features_df], axis=1)

    return extended_df


In [202]:
title = "Georgia off My Mind"
artist = ""
limit = 5

In [203]:
tracks=get_song_info_with_features(title, artist, limit)
tracks

song_name       artist  \
0                                Georgia Off My Mind   Larkin Poe   
1                                Georgia off My Mind   Jared Wade   
2  Georgia on My Mind (with Gene Krupa & His Orch...  Anita O'Day   

               spotify_id  danceability  energy  key  loudness  mode  \
0  4dLPPSxHgaoy38YmX4yJnV         0.643   0.599    2    -8.541     1   
1  61QrMlxsE6A0Qyw21RGmbZ         0.587   0.526    0    -6.738     1   
2  4dmyrJKIeq1Y2XYS48SDSA         0.571   0.302    7    -8.534     0   

   speechiness  acousticness  ...  liveness  valence    tempo            type  \
0       0.0292        0.0214  ...    0.2820    0.591  113.982  audio_features   
1       0.0245        0.4620  ...    0.1240    0.282  143.849  audio_features   
2       0.0341        0.9200  ...    0.0979    0.603  137.139  audio_features   

                       id                                   uri  \
0  4dLPPSxHgaoy38YmX4yJnV  spotify:track:4dLPPSxHgaoy38YmX4yJnV   
1  61QrMlxsE6A0Qyw21RGmbZ  spotify:track:61QrMlxsE6A0Qyw21RGmbZ   
2  4dmyrJKIeq1Y2XYS48SDSA  spotify:track:4dmyrJKIeq1Y2XYS48SDSA   

                                          track_href  \
0  https://api.spotify.com/v1/tracks/4dLPPSxHgaoy...   
1  https://api.spotify.com/v1/tracks/61QrMlxsE6A0...   
2  https://api.spotify.com/v1/tracks/4dmyrJKIeq1Y...   

                                        analysis_url duration_ms  \
0  https://api.spotify.com/v1/audio-analysis/4dLP...      234212   
1  https://api.spotify.com/v1/audio-analysis/61Qr...      234587   
2  https://api.spotify.com/v1/audio-analysis/4dmy...      174200   

   time_signature  
0               4  
1               4  
2               4  

[3 rows x 21 columns]

In [177]:
song_uris, song_df = search_song(title, artist, limit)
print("Song URIs:", song_uris)
print("n\Song DataFrame:")
print(song_df)


Song URIs: ['spotify:track:4dLPPSxHgaoy38YmX4yJnV', 'spotify:track:61QrMlxsE6A0Qyw21RGmbZ']
n\Song DataFrame:
             song_name      artist              spotify_id
0  Georgia Off My Mind  Larkin Poe  4dLPPSxHgaoy38YmX4yJnV
1  Georgia off My Mind  Jared Wade  61QrMlxsE6A0Qyw21RGmbZ


In [162]:
time.sleep(20)
audiofeats=get_audio_features(song_uris)
audiofeats

[{'danceability': 0.643,
  'energy': 0.599,
  'key': 2,
  'loudness': -8.541,
  'mode': 1,
  'speechiness': 0.0292,
  'acousticness': 0.0214,
  'instrumentalness': 1.89e-06,
  'liveness': 0.282,
  'valence': 0.591,
  'tempo': 113.982,
  'type': 'audio_features',
  'id': '4dLPPSxHgaoy38YmX4yJnV',
  'uri': 'spotify:track:4dLPPSxHgaoy38YmX4yJnV',
  'track_href': 'https://api.spotify.com/v1/tracks/4dLPPSxHgaoy38YmX4yJnV',
  'analysis_url': 'https://api.spotify.com/v1/audio-analysis/4dLPPSxHgaoy38YmX4yJnV',
  'duration_ms': 234212,
  'time_signature': 4},
 {'danceability': 0.643,
  'energy': 0.599,
  'key': 2,
  'loudness': -8.541,
  'mode': 1,
  'speechiness': 0.0292,
  'acousticness': 0.0214,
  'instrumentalness': 1.89e-06,
  'liveness': 0.282,
  'valence': 0.591,
  'tempo': 113.982,
  'type': 'audio_features',
  'id': '2se33uCsb88PVwbp6ESNGb',
  'uri': 'spotify:track:2se33uCsb88PVwbp6ESNGb',
  'track_href': 'https://api.spotify.com/v1/tracks/2se33uCsb88PVwbp6ESNGb',
  'analysis_url': 'ht

In [91]:
feature_keys = extract_audio_feature_keys(audiofeats)
print(feature_keys)

['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'type', 'id', 'uri', 'track_href', 'analysis_url', 'duration_ms', 'time_signature']


In [117]:
audio_feature_dict_dummy= {key: None for key in feature_keys}
print(audio_feature_dict_dummy)

{'danceability': None, 'energy': None, 'key': None, 'loudness': None, 'mode': None, 'speechiness': None, 'acousticness': None, 'instrumentalness': None, 'liveness': None, 'valence': None, 'tempo': None, 'type': None, 'id': None, 'uri': None, 'track_href': None, 'analysis_url': None, 'duration_ms': None, 'time_signature': None}


In [77]:
tracks=get_song_info_with_features(title, artist, limit)
tracks

Error: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))


Empty DataFrame
Columns: [song_name, artist, id, acousticness, danceability, energy]
Index: []